<a href="https://colab.research.google.com/github/dk-wei/ml-algo-implementation/blob/main/Random_Forest_Parameter_Tuning_%E8%B0%83%E5%8F%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考资料1：[Tuning the parameters of your Random Forest model](https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/)            
参考资料2：[sklearn.tree.DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)             
参考资料3：[InDepth: Parameter tuning for Decision Tree](https://medium.com/@mohtedibf/indepth-parameter-tuning-for-decision-tree-6753118a03c3#_=_)        
参考资料4：[scikit-learn随机森林类库概述](https://www.cnblogs.com/pinard/p/6160412.html)      


![](https://www.analyticsvidhya.com/wp-content/uploads/2015/06/random-forest7.png)


# RF重要的框架参数

首先我们关注于RF的Bagging框架的参数。这里可以和GBDT对比来学习。在scikit-learn 梯度提升树(GBDT)调参小结中我们对GBDT的框架参数做了介绍。GBDT的框架参数比较多，重要的有最大迭代器个数，步长和子采样比例，调参起来比较费力。但是RF则比较简单，**这是因为bagging框架里的各个弱学习器之间是没有依赖关系的，这减小的调参的难度**。换句话说，达到同样的调参效果，RF调参时间要比GBDT少一些。

下面我来看看RF重要的Bagging框架的参数，由于RandomForestClassifier和RandomForestRegressor参数绝大部分相同，这里会将它们一起讲，不同点会指出。

1. `n_estimators`: 也就是最大的弱学习器的个数，也可以说是树的个数。用来做maximium voting的依据。**一般来说n_estimators太小，容易欠拟合，n_estimators太大，容易过拟合**，计算量会太大，并且n_estimators到一定的数量后，再增大n_estimators获得的模型提升会很小，所以一般选择一个适中的数值。默认是100。

2. `oob_score` :即是否采用袋外样本来评估模型的好坏。是一种random forest的cross validation,  leave one out validation technique，但是要快很多。默认是False。个人推荐设置为True，因为袋外分数反应了一个模型拟合后的泛化能力。

3. `criterion`: 即CART树做划分时对特征的评价标准。分类模型和回归模型的损失函数是不一样的。分类RF对应的CART分类树默认是基尼系数gini,另一个可选择的标准是信息增益。回归RF对应的CART回归树默认是均方差mse，另一个可以选择的标准是绝对值差mae。一般来说选择默认的标准就已经很好的。

从上面可以看出， **RF重要的框架参数比较少，主要需要关注的是 n_estimators，即RF最大的决策树个数**。


![](https://www.analyticsvidhya.com/wp-content/uploads/2015/06/RF.png)

# RF决策树参数

Scikit-learn [参数说明文档](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

下面我们再来看RF的决策树参数，它要调参的参数基本和GBDT相同，如下:

1. **`max_features`**: RF创建每棵树划分时考虑的最大特征数，可以使用很多种类型的值，默认是"auto",意味着每次每棵树划分时最多考虑sqrt(N)个特征；如果是"log2"意味着划分时最多考虑log2N个特征；如果是"sqrt"或者"auto"意味着划分时最多考虑sqrt(N)个特征。如果是整数，代表考虑的特征绝对数。如果是浮点数，代表考虑特征百分比，即考虑（百分比xN）取整后的特征数。其中N为样本总特征数。一般我们用默认的"auto"就可以了，如果特征数非常多，我们可以灵活使用刚才描述的其他取值来控制划分时考虑的最大特征数，以控制决策树的生成时间。
  - Auto/None : This will simply take all the features which make sense in every tree.Here we simply do not put any restrictions on the individual tree.
  - sqrt : This option will take square root of the total number of features in individual run. For instance, if the total number of variables are 100, we can only take 10 of them in individual tree.”log2″ is another similar type of option for max_features.
  - 0.2 : This option allows the random forest to take 20% of variables in individual run. We can assign and value in a format “0.x” where we want x% of features to be considered.    
  
  **max_features越大，模型表现越好，但是速度可能越慢**。

2. **`max_depth`**: 决策树最大深度， 默认可以不输入，如果不输入的话，决策树在建立子树的时候不会限制子树的深度。一般来说，数据少或者特征少的时候可以不管这个值。如果模型样本量多，特征也多的情况下，推荐限制这个最大深度，具体的取值取决于数据的分布。**常用的可以取值10-100之间**。

3. **`min_samples_split`**: 内部node每次再划分所需最小样本阈值，这个值限制了子树继续划分的条件，如果某节点的样本数少于min_samples_split，也就是不够有区分度，则不会继续再尝试选择最优特征来进行划分。 可以是int也可以是float/fraction。**默认是2，接近于stamp. 如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值**。

4. **`min_samples_leaf`**:leaf node最少样本数， 这个值限制了leaf节点最少的样本数，如果某leaf node数目小于样本数，则会和兄弟节点一起被剪枝。 和上面的`min_samples_split`相似，也是sample 数量，但是区别是这个是最低端leaf的sample数量，leaf也就是base of tree。默认是1,可以输入最少的样本数的整数，或者最少样本数占样本总数的百分比。这个值越大，条件越苛刻，越小越容易split，容易overfitting。**如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值**。

5. `min_weight_fraction_leaf`：叶子节点最小的样本权重和，这个值限制了叶子节点所有样本权重和的最小值，如果小于这个值，则会和兄弟节点一起被剪枝。 默认是0，就是不考虑权重问题。一般来说，如果我们有较多样本有缺失值，或者分类树样本的分布类别偏差很大，就会引入样本权重，这时我们就要注意这个值了。

6. `max_leaf_nodes`: 最大叶子节点数，通过限制最大叶子节点数，可以防止过拟合，默认是"None”，即不限制最大的叶子节点数。如果加了限制，算法会建立在最大叶子节点数内最优的决策树。如果特征不多，可以不考虑这个值，但是如果特征分成多的话，可以加以限制，具体的值可以通过交叉验证得到。

7. `min_impurity_split`:  节点划分最小不纯度，这个值限制了决策树的增长，如果某节点的不纯度(基于基尼系数，均方差)小于这个阈值，则该节点不再生成子节点。即为叶子节点 。**一般不推荐改动默认值1e-7**。

上面决策树参数中最重要的包括
- 最大特征数`max_features`
- 最大深度`max_depth`
- 内部节点再划分所需最小样本数`min_samples_split`
- 叶子节点最少样本数`min_samples_leaf`

In [22]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn import metric

import matplotlib.pylab as plt
%matplotlib inline

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
train = pd.read_csv('/content/gdrive/MyDrive/扬FAANG起航/单项准备/rf调参/train_modified.csv')

target='Disbursed' # Disbursed的值就是二元分类的输出
IDcol = 'ID'
train['Disbursed'].value_counts() 

0    19680
1      320
Name: Disbursed, dtype: int64

In [6]:
x_columns = [x for x in train.columns if x not in [target, IDcol]]
X = train[x_columns]
y = train['Disbursed']

用默认参数，我们拟合下数据看看，输出如下，可见袋外分数已经很高，而且AUC分数也很高。相对于GBDT的默认参数输出，RF的默认参数拟合效果对本例要好一些。

In [23]:
rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(X,y)
print (rf0.oob_score_)
y_predprob = rf0.predict_proba(X)[:,1]
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))

0.98315
AUC Score (Train): 0.999994


我们可以先weak learnder，也就是决策树的大概的个数，然后再调整具体数的参数，例如max feature还有max depth.

我们首先对`n_estimators`进行网格搜索：

In [44]:
param_test1 = {'n_estimators':range(10,71,10)}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt' ,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',cv=5)
gsearch1.fit(X,y)

gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([0.09737368, 0.18666906, 0.26725407, 0.35473409, 0.435953  ,
         0.52016726, 0.61556268]),
  'mean_score_time': array([0.00979362, 0.0136826 , 0.01808734, 0.02235842, 0.02677369,
         0.03250804, 0.03588705]),
  'mean_test_score': array([0.80680934, 0.81600252, 0.81818272, 0.81838438, 0.82034069,
         0.82113345, 0.8199191 ]),
  'param_n_estimators': masked_array(data=[10, 20, 30, 40, 50, 60, 70],
               mask=[False, False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 10},
   {'n_estimators': 20},
   {'n_estimators': 30},
   {'n_estimators': 40},
   {'n_estimators': 50},
   {'n_estimators': 60},
   {'n_estimators': 70}],
  'rank_test_score': array([7, 6, 5, 4, 2, 1, 3], dtype=int32),
  'split0_test_score': array([0.81797431, 0.82673558, 0.8370927 , 0.83676321, 0.8351753 ,
         0.83643769, 0.83286093]),
  'split1_test_score': array([0.78064461, 0.78217893, 0.79100

In [27]:
gsearch1.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

这样我们得到了最佳的弱学习器迭代次数，接着我们对决策树最大深度`max_depth`和内部节点再划分所需最小样本数`min_samples_split`进行网格搜索。

In [38]:
param_test2 = {'max_depth':range(3,14,2), 'min_samples_split':range(50,201,20)}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60, 
                                  min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=10),
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.cv_results_, gsearch1.best_params_, gsearch1.best_score_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([0.48619204, 0.47403102, 0.47277865, 0.47762794, 0.48582964,
         0.48572745, 0.48395581, 0.49025869, 0.59677639, 0.604145  ,
         0.58711305, 0.59297543, 0.58015485, 0.58319831, 0.58127055,
         0.59670105, 0.67404227, 0.68936887, 0.67041025, 0.67467566,
         0.67974687, 0.66841412, 0.65125151, 0.64449878, 0.71106262,
         0.70772796, 0.71869864, 0.72425251, 0.72019491, 0.71184611,
         0.70741677, 0.72220387, 0.76552558, 0.74090276, 0.74559498,
         0.73648586, 0.72177835, 0.71784348, 0.71682892, 0.7128736 ,
         0.76107426, 0.74600258, 0.7728457 , 0.74494867, 0.73823733,
         0.74819303, 0.76283293, 0.72680211]),
  'mean_score_time': array([0.02452073, 0.02415905, 0.02410793, 0.02356906, 0.02713475,
         0.02343044, 0.02502065, 0.02479644, 0.02853055, 0.02848873,
         0.02829356, 0.02716832, 0.02699685, 0.02825875, 0.02812152,
         0.02851152, 0.03091474, 0.03279233, 0.03192153, 0.03004618,
         0.03174796,

In [40]:
rf1 = RandomForestClassifier(n_estimators= 60, max_depth=13, min_samples_split=110,
                                  min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=10)
rf1.fit(X,y)
print (rf1.oob_score_)

0.984


可见此时我们的袋外分数有一定的提高。也就是时候模型的泛化能力增强了。


对于内部节点再划分所需最小样本数`min_samples_split`，我们暂时不能一起定下来，因为这个还和决策树其他的参数存在关联。下面我们再对内部节点再划分所需最小样本数`min_samples_split`和叶子节点最少样本数`min_samples_leaf`一起调参。

In [41]:
param_test3 = {'min_samples_split':range(80,150,20), 
               'min_samples_leaf':range(10,60,10)
               }
               
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60, max_depth=13,
                                  max_features='sqrt' ,oob_score=True, random_state=10),
   param_grid = param_test3, scoring='roc_auc',iid=False, cv=5)
gsearch3.fit(X,y)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([0.77233701, 0.76798534, 0.75996361, 0.76072397, 0.76553292,
         0.7545238 , 0.75625334, 0.74983978, 0.7427434 , 0.74143205,
         0.74071693, 0.72180557, 0.73838181, 0.72688351, 0.71862154,
         0.7137569 , 0.69695363, 0.70479155, 0.71587658, 0.70639639]),
  'mean_score_time': array([0.03583164, 0.03525195, 0.03656549, 0.03589606, 0.03482056,
         0.03518543, 0.03468833, 0.03647327, 0.0364234 , 0.0352963 ,
         0.03546715, 0.03372583, 0.03411355, 0.03382416, 0.0339633 ,
         0.03549223, 0.03443851, 0.03419318, 0.03415442, 0.03560438]),
  'mean_test_score': array([0.8209294 , 0.81913348, 0.82048399, 0.8179751 , 0.8209429 ,
         0.82097426, 0.82486503, 0.82169239, 0.82352087, 0.82164475,
         0.82069876, 0.82141332, 0.82278249, 0.82141411, 0.82042881,
         0.82162093, 0.82224975, 0.82224975, 0.81890403, 0.81916643]),
  'param_min_samples_leaf': masked_array(data=[10, 10, 10, 10, 20, 20, 20, 20, 30, 30, 30, 30, 40, 40,
        

In [42]:
rf2 = RandomForestClassifier(n_estimators= 60, max_depth=13, min_samples_split=120,
                                  min_samples_leaf=20,max_features=7 ,oob_score=True, random_state=10)
rf2.fit(X,y)
print (rf2.oob_score_)

0.984


最后我们再对最大特征数`max_features`做调参:

In [45]:
param_test4 = {'max_features':range(3,11,2)}
gsearch4 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60, max_depth=13, min_samples_split=120,
                                  min_samples_leaf=20 ,oob_score=True, random_state=10),
   param_grid = param_test4, scoring='roc_auc',iid=False, cv=5)
gsearch4.fit(X,y)
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'mean_fit_time': array([0.56057458, 0.65641112, 0.77123132, 0.84812627]),
  'mean_score_time': array([0.03341579, 0.03474703, 0.03478885, 0.03456774]),
  'mean_test_score': array([0.81981191, 0.8163868 , 0.82486503, 0.81703506]),
  'param_max_features': masked_array(data=[3, 5, 7, 9],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_features': 3},
   {'max_features': 5},
   {'max_features': 7},
   {'max_features': 9}],
  'rank_test_score': array([2, 4, 1, 3], dtype=int32),
  'split0_test_score': array([0.81893102, 0.82697972, 0.83293834, 0.81775994]),
  'split1_test_score': array([0.79912387, 0.79626763, 0.79838748, 0.80414563]),
  'split2_test_score': array([0.78474935, 0.77782211, 0.80341916, 0.78332023]),
  'split3_test_score': array([0.84169565, 0.83561793, 0.8346374 , 0.83346434]),
  'split4_test_score': array([0.85455967, 0.8452466 , 0.85494276, 0.84648517]),
  'std_fit_time': array([0.00943765, 0.0164145

　用我们搜索到的最佳参数，我们再看看最终的模型拟合：

In [46]:
rf2 = RandomForestClassifier(n_estimators= 60, max_depth=13, min_samples_split=120,
                                  min_samples_leaf=20,max_features=7 ,oob_score=True, random_state=10)
rf2.fit(X,y)
print (rf2.oob_score_)

0.984


　可见此时模型的袋外分数基本没有提高，主要原因是0.984已经是一个很高的袋外分数了，如果想进一步需要提高模型的泛化能力，我们需要更多的数据。